In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from scipy import linalg

# prox. l1 norm (soft thresholding function)
def soft(z, th):
    return np.sign(z) * np.maximum(np.abs(z) - th, 0)

# prox. ||.-y||_2**2
def prox_squ_l2(q, l=1, c=None):
    if c is None:
        return q / (l+1.)
    else:
        return (q + l * c) / (l+1.)

# prox. nuclear norm (singular value thresholding)
def prox_nuclear(Z, th):
    U, sv, Vh = linalg.svd(Z,full_matrices=False)
    sv = soft(sv, th)
    r = np.count_nonzero(sv)
    U = U[:,:r]
    sv = sv[:r]
    Vh = Vh[:r,:]
    return U.dot(sp.diags(sv).dot(Vh)), U, sv, Vh

In [ ]:
from scipy import sparse as sp

# Low rank matrix completion by ADMM
def LowRankMatrixCompletion(Y, l=1, rho=1., maxiter=300, rtol=1e-6, verbose=False):
    Y = np.ma.masked_invalid(Y)
    numObsY = Y.count()
    # Y and R are modified to data and mask arrays, respectively.
    Y, R = Y.data, ~Y.mask

    m, n = Y.shape
    G = sp.vstack((sp.eye(m*n, format='csr', dtype=Y.dtype)[R.ravel()], sp.eye(m*n, dtype=Y.dtype), sp.eye(m*n, dtype=Y.dtype)))
    # pinvG = linalg.pinv(G.toarray())
    # Pseudo inverse of G is explicitly described as 
    pinvG = 0.5 * np.ones(m*n, dtype=Y.dtype)
    pinvG[R.ravel()] = 1./3.
    pinvG = sp.diags(pinvG, format='csr') # sp.dia_matrix((pinvG,np.array([0])), shape=())
    pinvG = sp.hstack(((1./3.)*sp.eye(m*n, format='csr', dtype=Y.dtype)[R.ravel()].T, pinvG, pinvG))
    
    # initialize
    x = np.zeros(m*n, dtype=Y.dtype)
    z = np.zeros(numObsY+2*m*n, dtype=Y.dtype)
    u = np.zeros_like(z) #np.zeros(z.shape, dtype=z.dtype)
    count = 0
    cost_history = []

    while count < maxiter:        
        x = pinvG.dot(z - u)
 
        Gx = G.dot(x)
        q = Gx + u

        dz = z.copy()
        z[:numObsY] = prox_squ_l2(q[:numObsY], 1/rho, c=Y[R].ravel()) # (rho*q[:numObsY] + Y[R].ravel())/(rho+1.)
        L, U, sv, Vh = prox_nuclear(q[numObsY:numObsY+m*n].reshape(m,n), l/rho)
        z[numObsY:numObsY+m*n] = L.ravel()
        q = q[numObsY+m*n:];  q[q<0.] = 0.
        z[numObsY+m*n:] = q
        dz = z - dz

        du = u.copy()
        u = u + Gx - z
        du = u - du

        normr2 = linalg.norm(x[R.ravel()] - Y[R].ravel())**2
        tr = np.sum(sv)
        cost_history.append( 0.5*normr2 + l*tr )
        if verbose:
            if np.fmod(count,verbose) == 0:
                print('%2d: 0.5*||R*(Y-A)||_F^2 + l*||A||_* = %.2e + %.2e = %.2e' 
                      % (count, 0.5*normr2, l*tr, 0.5*normr2+l*tr))

        # check convergence of primal and dual residuals
        if linalg.norm(du) < rtol * linalg.norm(u) and linalg.norm(dz) < rtol * linalg.norm(z):
            break
        if count > 0 and abs(cost_history[count-1] - cost_history[count]) < rtol*cost_history[count-1]:
            break
        count += 1
    return x.reshape(m,n), U, sv, Vh, count

In [ ]:
#Y = np.array([[     4,      0,      5,      1],
#              [     3,      0,      4,      1],
#              [     0,      5,      0,      4],
#              [     4,      0,      5,      0],
#              [     0,      4,      0,      3]])
Y = np.array([[     4,      0,      5,      1],
              [np.nan,      0,      4, np.nan],
              [     0,      5,      0,      4],
              [     4, np.nan, np.nan,      0],
              [     0,      4,      0, np.nan]])
print('Y = ')
print(Y)

In [ ]:
from time import time
print("Running LowRankMatrixCompletion..")
t0 = time()
Yest, _, s, _, it = LowRankMatrixCompletion(Y, l=1., verbose=10)
print('done in %.2fs with %d steps' % (time() - t0, it))
print('rel. error = %.2e' % (linalg.norm((Yest-Y)[~np.isnan(Y)])/linalg.norm(Y[~np.isnan(Y)])))
np.set_printoptions(precision=3, suppress=True)
print('Yest = ')
print(Yest)
print('sv = ')
print(s)